In [71]:
from reportlab.lib import colors

input_pdf = "input/combined.pdf"
output_pdf = "output/numbered.pdf"
page_number_color=colors.darkred
background_color=colors.Color(255, 255, 255, alpha=0.5)

In [72]:
from PyPDF2 import PdfWriter, PdfReader
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

import io

input_pdf_reader = PdfReader(open(input_pdf, "rb"))
output_pdf_writer = PdfWriter()
total_pages = len(input_pdf_reader.pages)

# get the dimensions of the first page
# and calculate font size based its size
page = input_pdf_reader.pages[0]
page_width = page.mediabox.width
font_size = float(page_width) * 0.025  # 2% of the smaller dimension
text_height = font_size

for page_num in range(total_pages):
    packet = io.BytesIO()
    can = canvas.Canvas(packet, pagesize=letter)

    # calculate text width
    page_num_str = f'{page_num + 1}/{total_pages}'
    text_width = can.stringWidth(page_num_str, "Courier", font_size)
    
    # add rectangle background
    can.setFillColor(background_color)
    can.rect(10 - 2, 10 - 2, text_width + 4, text_height, fill=1, stroke=0)

    # add page number
    can.setFillColor(page_number_color)  # set the color of the page number
    can.setFont("Courier", font_size)  # Set the font size
    can.drawString(10, 10, page_num_str)  # position the page number
    can.save()

    # move to the beginning of the StringIO buffer
    packet.seek(0)
    page_number_pdf = PdfReader(packet)

    # merge the page number pdf with the input pdf page
    page = input_pdf_reader.pages[page_num]
    page.merge_page(page_number_pdf.pages[0])
    output_pdf_writer.add_page(page)

with open(output_pdf, "wb") as output_file:
    output_pdf_writer.write(output_file)